In [1]:
import pandas as pd

from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np

from surprise import SVD, NMF
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from helpers import *

In [2]:
data = load_data("data/data_train.csv")
data.head()

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


In [ ]:
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))
param_grid = {'n_epochs': [10, 20], 'n_factors': [20, 40], 
              'lr_bu': [0.002, 0.008], 'lr_bi': [0.002, 0.008],
              'reg_bu': [0.03, 0.06], 'reg_bi': [0.03, 0.06],
             'reg_pu': [0.03, 0.06], 'reg_qi': [0.03, 0.06]}
# param_grid = {'n_epochs': [10, 30], 'lr_all': [0.0005, 0.003], 'reg_all': [0.1, 0.4]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=None, n_jobs=-1, refit=True, joblib_verbose=10)
gs.fit(dataset)
# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  8.0min
/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 10.6min


In [ ]:
def search_params(params, dataset):
    gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=2, n_jobs=-1, refit=True, joblib_verbose=10)
    gs.fit(dataset)
    print(gs.best_score['rmse'])
    print(gs.best_params['rmse'])
    return gs

In [ ]:
gs2 = search_params({
    'n_epochs': [10, 20], 'n_factors': [20, 40], 
    'lr_bu': [0.002, 0.008], 'lr_bi': [0.002, 0.008],
    'reg_bu': [0.03, 0.06], 'reg_bi': [0.03, 0.06],
    'reg_pu': [0.03, 0.06], 'reg_qi': [0.03, 0.06] 
}, dataset)

In [ ]:
submission = pd.read_csv("data/sample_submission.csv")

In [ ]:
def predict(user, movie):
    return int(round(recommenders[cluster_dict[movie]].predict(user, movie).est))

In [ ]:
submission['Prediction'] = [int(round(gs.best_estimator['rmse'].predict(user, movie).est)) for [user, movie] in submission['Id'].str.split('_')]
submission.to_csv(r'data/nmf_actual.csv', index=False)